In [1]:
%%capture
pip install mne

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import mne
import pywt

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%%capture
# folder = 'E:/Data WareHouse/1sec/Epileptic Seizure/equalized'
folder = '/content/drive/MyDrive/EEG Signal /Epileptic seizure/data/equalized epoch'
epochs_path = [os.path.join(folder,i) for i in os.listdir(folder) if i[-3:]=='fif']

In [5]:
%%capture
# folder = 'E:/Data WareHouse/1sec/Epileptic Seizure/equalized'
# folder = '/content/drive/MyDrive/EEG Signal /Epileptic seizure/data/equalized epoch'
# epochs_path = [os.path.join(folder,i) for i in os.listdir(folder) if i[-3:]=='fif']

data= [mne.read_epochs(i).pick_types(eeg=True) for i in epochs_path]
labels = [mne.read_epochs(i).events[:,2] for i in epochs_path]
group = [[i]*len(j) for i,j in enumerate(data)]
X=np.vstack(data)
Y=np.hstack(labels)
group= np.hstack(group)

In [6]:
print(X.shape,Y.shape,group.shape)

(5834, 29, 180) (5834,) (5834,)


# Feature Extraction methods

##Time-domain Features

1. Mean (mean_val): The average value of the signal amplitude, representing the signal's central tendency.

2. Median (median_val): The middle value of the signal amplitude when sorted, representing the central tendency of the signal, less sensitive to outliers than the mean.

3. Variance (var_val): Measures the spread of the signal's amplitude.

4. Standard Deviation (std_dev): The square root of the variance, representing the dispersion of the signal's amplitude.

5. Skewness (skewness): Measures the asymmetry of the signal's amplitude distribution around the mean.

6. Kurtosis (kurt): Measures the 'tailedness' of the signal's amplitude distribution, indicating the presence of outliers.

7. Zero Crossing Rate (zcr): The rate at which the signal changes sign, indicating frequency content.

8. Root Mean Square (rms_val): Represents the square root of the average of the squares of the signal, indicative of the signal's magnitude.

9. Signal Energy (energy): The sum of the squares of the signal values, indicative of the signal's power.

10. Crest Factor (crest_fact): The ratio of the peak amplitude of the waveform to the RMS value, indicating the extremeness of peaks.

11. Shape Factor (shape_fact): The ratio of the RMS value to the mean absolute value, indicative of the waveform shape.

12. Entropy (signal_entropy): Measures the unpredictability or complexity of the signal.

13. Peak Amplitude (peak_amp): The difference between the maximum and minimum amplitude, indicating the signal's range.

14. Number of Peaks (num_peaks): The count of local maxima, indicating the frequency of oscillations.

15. Average Peak-to-Peak Distance (peak_to_peak_distance): The average distance between consecutive peaks, related to the periodicity of the signal.

16. Hjorth Parameters:

  Activity: Indicates the signal power.

  Mobility: Indicates the mean frequency or the proportion of the standard deviation of the power spectrum.
  
  Complexity: Indicates the bandwidth of the signal or the change in frequency.





In [ ]:
# import numpy as np
# from scipy.stats import skew, kurtosis
# from scipy.signal import find_peaks

# def zero_crossing_rate(signal):
#     zero_crossings = np.where(np.diff(np.sign(signal)))[0]
#     return len(zero_crossings) / len(signal)

# def hjorth_parameters(signal):
#     diff_input = np.diff(signal)
#     diff_diff_input = np.diff(diff_input)

#     activity = np.var(signal)
#     mobility = np.sqrt(np.var(diff_input)/activity)
#     complexity = np.sqrt(np.var(diff_diff_input)/np.var(diff_input)) / mobility

#     return activity, mobility, complexity




# def extract_time_domain_features(epochs):
#     features = []

#     for epoch in epochs:
#         epoch_features = []

#         for channel_data in epoch:
#             # Flatten the channel data
#             flattened_data = channel_data.flatten()

#             # Basic Time-Domain Features
#             mean_val = np.mean(flattened_data)
#             median_val = np.median(flattened_data)
#             var_val = np.var(flattened_data)
#             std_dev = np.std(flattened_data)
#             skewness = skew(flattened_data)
#             kurt = kurtosis(flattened_data)
#             zcr = zero_crossing_rate(flattened_data)
#             peak_amp = np.ptp(flattened_data)

#             # Hjorth Parameters
#             activity, mobility, complexity = hjorth_parameters(flattened_data)

#             # Additional Features
#             num_waves = len(find_peaks(flattened_data)[0])
#             wave_duration = len(flattened_data) / num_waves if num_waves > 0 else 0

#             channel_features = [
#                 mean_val, median_val, var_val, std_dev, skewness, kurt, zcr, num_waves,
#                 wave_duration, peak_amp, activity, mobility, complexity
#             ]
#             epoch_features.append(channel_features)

#         features.append(epoch_features)

#     return np.array(features)

In [7]:
import numpy as np
from scipy.stats import skew, kurtosis, entropy
from scipy.signal import find_peaks


def hjorth_parameters(signal):
    diff_input = np.diff(signal)
    diff_diff_input = np.diff(diff_input)

    activity = np.var(signal)
    mobility = np.sqrt(np.var(diff_input)/activity)
    complexity = np.sqrt(np.var(diff_diff_input)/np.var(diff_input)) / mobility

    return activity, mobility, complexity

# def zero_crossing_rate(signal):
#     # Enhanced ZCR to handle noise
#     # Setting a threshold (eps) to consider as zero (to avoid detecting false crossings due to noise)
#     eps = 0.01 * np.std(signal)
#     zero_crossings = np.where(np.diff(np.sign(signal)))[0]
#     close_to_zero = np.where(np.abs(signal) < eps)[0]
#     return (len(zero_crossings) + len(close_to_zero)) / len(signal)

def zero_crossing_rate(signal):
    zero_crossings = np.where(np.diff(np.sign(signal)))[0]
    return len(zero_crossings) / len(signal)

def safe_divide(numerator, denominator, default=0.0):
    """Safely divide two numbers, returning a default value if the denominator is zero."""
    if denominator == 0:
        return default
    else:
        return numerator / denominator

def signal_entropy(signal):
    # Calculate histogram of the signal
    hist, bin_edges = np.histogram(signal, bins='auto', density=True)
    # Ensure non-zero histogram values by adding a small constant
    hist += np.finfo(float).eps
    # Normalize the histogram to get a probability distribution
    prob_dist = hist / hist.sum()
    # Calculate the entropy
    return entropy(prob_dist)

def rms(signal):
    return np.sqrt(np.mean(signal**2))

def signal_energy(signal):
    return np.sum(signal**2)



def crest_factor(signal, rms_val=None):
    if rms_val is None:
        rms_val = rms(signal)
    return np.max(np.abs(signal)) / rms_val

def shape_factor(signal, rms_val=None):
    if rms_val is None:
        rms_val = rms(signal)
    mean_abs = np.mean(np.abs(signal))
    return rms_val / mean_abs if mean_abs != 0 else 0

def extract_time_domain_features(epochs):
    features = []

    for epoch in epochs:
        epoch_features = []

        for channel_data in epoch:
            # Flatten the channel data if needed
            flattened_data = np.ravel(channel_data)

            # Basic Time-Domain Features
            mean_val = np.mean(flattened_data)
            median_val = np.median(flattened_data)
            var_val = np.var(flattened_data)
            std_dev = np.std(flattened_data)
            skewness = skew(flattened_data)
            kurt = kurtosis(flattened_data)
            zcr = zero_crossing_rate(flattened_data)
            rms_val = rms(flattened_data)
            energy = signal_energy(flattened_data)
            crest_fact = crest_factor(flattened_data, rms_val)
            shape_fact = shape_factor(flattened_data, rms_val)
            signal_entropy_val = signal_entropy(flattened_data)  # Use the defined function
            peak_amp = np.ptp(flattened_data)

            # Hjorth Parameters
            activity, mobility, complexity = hjorth_parameters(flattened_data)

            # Additional Features
            num_peaks = len(find_peaks(flattened_data)[0])
            peak_to_peak_distance = np.mean(np.diff(find_peaks(flattened_data)[0])) if num_peaks > 1 else 0

            channel_features = [
                mean_val, median_val, var_val, std_dev, skewness, kurt, zcr, rms_val, energy,
                 crest_fact, shape_fact, signal_entropy_val,
                peak_amp, num_peaks, peak_to_peak_distance,
                activity, mobility, complexity
            ]
            epoch_features.append(channel_features)

        features.append(epoch_features)

    return np.array(features)

## Frequency Domain Features


1. Power Spectral Density (PSD) Calculation:

Utilizes the welch method to compute the power spectral density of the signal, which forms the basis for many of the following features.

2. Basic Statistical Features from PSD:

  Mean (mean_val): The average power in the spectrum.

*   Median (median_val): The middle value of the power spectrum.
*   Variance (var_val): The variance of the power spectrum.
*   Standard Deviation (std_dev): The standard deviation of the power spectrum.
*   Standard Deviation (std_dev): The standard deviation of the power spectrum.
* Skewness (skewness): A measure of the asymmetry of the power spectrum.
* Kurtosis (kurt): A measure of the tailedness of the power spectrum.

.

3. Wavelet Coefficients:

* Wavelet Coefficients (wave_coeffs): Computed using the Discrete Wavelet
* Transform (DWT) to provide a multi-resolution analysis of the signal.

* Mean of Wavelet Coefficients (wave_coeffs_mean): The mean of the wavelet coefficients, providing a summary of the wavelet-transformed data.

4. Band Power Features:

Computed by summing the PSD within specified frequency bands. The bands considered are delta, theta, alpha, beta, gamma, and sigma.


5. Band Power Ratios:

* Theta/Alpha Ratio (theta_alpha_ratio): The ratio of power in the theta band to the power in the alpha band.

* Beta/Alpha Ratio (beta_alpha_ratio): The ratio of power in the beta band to the power in the alpha band.

* (Theta + Alpha)/Beta Ratio (theta_alpha_beta_ratio): The ratio of the sum of power in the theta and alpha bands to the power in the beta band.

* Theta/Beta Ratio (theta_beta_ratio): The ratio of power in the theta band to the power in the beta band.

* (Theta + Alpha)/(Alpha + Beta) Ratio (theta_alpha_beta_alpha_ratio): The ratio of the sum of power in the theta and alpha bands to the sum of power in the alpha and beta bands.

* Gamma/Delta Ratio (gamma_delta_ratio): The ratio of power in the gamma band to the power in the delta band.

* (Gamma + Beta)/(Delta + Alpha) Ratio (gamma_beta_delta_alpha_ratio): The ratio of the sum of power in the gamma and beta bands to the sum of power in the delta and alpha bands.

6. Additional Frequency Domain Features:

* Spectral Entropy (spectral_entropy_val): Measures the entropy of the power spectrum, providing an index of the complexity or disorder in the frequency domain.

* Spectral Edge Frequency (spectral_edge_freq): The frequency below which a certain percentage (e.g., 95%) of the power of the signal is contained, providing a cutoff frequency that encloses most of the signal's power.

In [ ]:
# import numpy as np
# from scipy.stats import skew, kurtosis
# from scipy.signal import welch



# def get_wavelet_coeffs(channel_data, wavelet='db4', level=5):
#     coeffs = pywt.wavedec(channel_data, wavelet, level=level)
#     return coeffs


# def extract_frequency_domain_features(epochs, sfreq,wavelet='db4', bands={'delta': (1.59, 4), 'theta': (4, 8), 'alpha': (8, 12), 'beta': (12, 30), 'gamma': (30, 100), 'sigma': (11, 16)}):
#     features = []

#     for epoch in epochs:
#         epoch_features = []

#         for channel_data in epoch:
#             # Compute the Power Spectral Density (PSD)
#             freqs, psd = welch(channel_data, sfreq, nperseg=256)

#             # Frequency domain features
#             mean_val = np.mean(psd)
#             median_val = np.median(psd)
#             var_val = np.var(psd)
#             std_dev = np.std(psd)
#             skewness = skew(psd)
#             kurt = kurtosis(psd)

#             # Compute wavelet coefficients
#             wave_coeffs = get_wavelet_coeffs(channel_data, wavelet, level=5)
#             wave_coeffs_mean = np.mean(wave_coeffs[0])

#             # Band Power Features
#             band_powers = {}
#             for band, freq_range in bands.items():
#                 freq_mask = (freqs >= freq_range[0]) & (freqs <= freq_range[1])
#                 band_power = np.sum(psd[freq_mask])
#                 band_powers[band] = band_power

#             # Band Power Ratios
#             theta_alpha_ratio = band_powers['theta'] / band_powers['alpha']
#             beta_alpha_ratio = band_powers['beta'] / band_powers['alpha']
#             theta_alpha_beta_ratio = (band_powers['theta'] + band_powers['alpha']) / band_powers['beta']

#             # Additional Band Power Ratios
#             theta_beta_ratio = band_powers['theta'] / band_powers['beta']
#             theta_alpha_beta_alpha_ratio = (band_powers['theta'] + band_powers['alpha']) / (band_powers['alpha'] + band_powers['beta'])
#             gamma_delta_ratio = band_powers['gamma'] / band_powers['delta']
#             gamma_beta_delta_alpha_ratio = (band_powers['gamma'] + band_powers['beta']) / (band_powers['delta'] + band_powers['alpha'])


#             channel_features = [
#                 mean_val, median_val, var_val, std_dev, skewness, kurt,
#                 band_powers['delta'], band_powers['theta'], band_powers['alpha'],
#                 band_powers['beta'], band_powers['gamma'], band_powers['sigma'],
#                 theta_alpha_ratio, beta_alpha_ratio, theta_alpha_beta_ratio,theta_beta_ratio,
#                 theta_alpha_beta_alpha_ratio, gamma_delta_ratio, gamma_beta_delta_alpha_ratio,
#                 wave_coeffs_mean
#             ]
#             epoch_features.append(channel_features)

#         features.append(epoch_features)

#     return np.array(features)

In [8]:
import numpy as np
import pywt
from scipy.stats import skew, kurtosis, entropy
from scipy.signal import welch

def get_wavelet_coeffs(channel_data, wavelet='db4', level=5):
    coeffs = pywt.wavedec(channel_data, wavelet, level=level)
    return coeffs


def safe_divide(numerator, denominator, default_value=0.0):
    """Safely divide two numbers, returning a default value if the denominator is zero."""
    if denominator == 0:
        return default_value
    else:
        return numerator / denominator



def spectral_entropy(psd, sfreq):
    # Adding a small constant to avoid division by zero when normalizing
    psd_sum = np.sum(psd) + np.finfo(float).eps
    psd_norm = psd / psd_sum
    # Normalizing PSD to a probability distribution
    psd_norm = psd_norm / np.sum(psd_norm)
    # Ensuring the normalized PSD values are positive
    psd_norm[psd_norm <= 0] = np.finfo(float).eps
    return entropy(psd_norm)


def spectral_edge_frequency(freqs, psd, edge_percent=0.95):
    psd_sum = np.sum(psd) + np.finfo(float).eps  # Add a small constant to ensure non-zero denominator
    psd_cumsum = np.cumsum(psd) / psd_sum
    idx = np.where(psd_cumsum <= edge_percent)[0][-1] if len(psd_cumsum) > 0 else 0
    spectral_edge_freq = freqs[idx] if idx < len(freqs) else 0
    return spectral_edge_freq

def extract_frequency_domain_features(epochs, sfreq, wavelet='db4', bands={'delta': (1.59, 4), 'theta': (4, 8), 'alpha': (8, 12), 'beta': (12, 30), 'sigma': (11, 16)}):
    features = []

    for epoch in epochs:
        epoch_features = []

        for channel_data in epoch:
            # Compute the Power Spectral Density (PSD)
            freqs, psd = welch(channel_data, sfreq, nperseg=256)

            # Frequency domain features
            mean_val = np.mean(psd)
            median_val = np.median(psd)
            var_val = np.var(psd)
            std_dev = np.std(psd)
            skewness = skew(psd)
            kurt = kurtosis(psd)

            # Compute wavelet coefficients
            wave_coeffs = get_wavelet_coeffs(channel_data, wavelet, level=5)
            wave_coeffs_mean = np.mean(wave_coeffs[0])

            # Band Power Features
            band_powers = {}
            for band, freq_range in bands.items():
                freq_mask = (freqs >= freq_range[0]) & (freqs <= freq_range[1])
                band_power = np.sum(psd[freq_mask])
                band_powers[band] = band_power

            # Band Power Ratios
            # Band Power Ratios with safe division
            theta_alpha_ratio = safe_divide(band_powers['theta'], band_powers['alpha'])
            beta_alpha_ratio = safe_divide(band_powers['beta'], band_powers['alpha'])
            theta_alpha_beta_ratio = safe_divide(band_powers['theta'] + band_powers['alpha'], band_powers['beta'])
            theta_beta_ratio = safe_divide(band_powers['theta'], band_powers['beta'])
            theta_alpha_beta_alpha_ratio = safe_divide(band_powers['theta'] + band_powers['alpha'], band_powers['alpha'] + band_powers['beta'])

            # gamma_delta_ratio = band_powers['gamma'] / band_powers['delta']
            # gamma_beta_delta_alpha_ratio = (band_powers['gamma'] + band_powers['beta']) / (band_powers['delta'] + band_powers['alpha'])

            # Additional Features
            spectral_entropy_val = spectral_entropy(psd, sfreq)
            spectral_edge_freq = spectral_edge_frequency(freqs, psd)

            channel_features = [
                mean_val, median_val, var_val, std_dev, skewness, kurt,
                band_powers['delta'], band_powers['theta'], band_powers['alpha'],
                band_powers['beta'], band_powers['sigma'],
                theta_alpha_ratio, beta_alpha_ratio, theta_alpha_beta_ratio, theta_beta_ratio,
                theta_alpha_beta_alpha_ratio,
                wave_coeffs_mean, spectral_entropy_val, spectral_edge_freq
            ]
            epoch_features.append(channel_features)

        features.append(epoch_features)

    return np.array(features)

## Mel-frequency cepstrum coefficients (MFCCs)

In [21]:
!pip install librosa

In [22]:
import numpy as np
import librosa

def extract_eeg_mfcc_features(eeg_epochs, sr, n_mfcc=13):
    """
    Extract MFCC features from EEG signal epochs.

    Parameters:
    - eeg_epochs: A list (or an array) of EEG epochs, where each epoch is a time series of signal data from one or more channels.
    - sr: Sampling rate of the EEG signal.
    - n_mfcc: Number of MFCC features to extract. Default is 13, but consider adjusting for EEG data.

    Returns:
    - A numpy array of MFCC features. Shape: (number of epochs, number of channels, n_mfcc).
    """
    mfcc_features = []

    for epoch in eeg_epochs:
        epoch_features = []
        for channel_data in epoch:
            # Compute MFCCs from EEG channel data
            mfcc = librosa.feature.mfcc(y=channel_data, sr=sr, n_mfcc=n_mfcc)

            # Compute mean and standard deviation as a simple way to summarize the MFCCs over time
            mfcc_mean = np.mean(mfcc, axis=1)
            mfcc_std = np.std(mfcc, axis=1)

            # Combine mean and standard deviation into a single feature vector for the channel
            channel_features = np.concatenate([mfcc_mean, mfcc_std])
            epoch_features.append(channel_features)

        mfcc_features.append(epoch_features)

    return np.array(mfcc_features)

In [24]:
X_mfcc = extract_eeg_mfcc_features(X,256)

/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=180
  warnings.warn(


In [25]:
X_mfcc.shape

(5834, 29, 26)

#Apply Feature Extraction Methods

In [9]:
X_time = extract_time_domain_features(X)
sfreq = 256  # Replace with the sampling frequency of your data
# epochs_data = [epoch.get_data() for epoch in epochs]  # Assuming epochs is a list of MNE Epochs objects
X_frequency = extract_frequency_domain_features(X, sfreq)

/usr/local/lib/python3.10/dist-packages/scipy/signal/_spectral_py.py:2017: UserWarning: nperseg = 256 is greater than input length  = 180, using nperseg = 180
  warnings.warn('nperseg = {0:d} is greater than input length '
/usr/local/lib/python3.10/dist-packages/pywt/_multilevel.py:43: UserWarning: Level value of 5 is too high: all coefficients will experience boundary effects.
  warnings.warn(


In [10]:
#merge the time and frequency features
X_merged_features = np.concatenate((X_time ,X_frequency), axis=2)

X_reshape = X_merged_features.reshape(X_merged_features.shape[0], -1)
# X_reshape = DWT_data.reshape(DWT_data.shape[0], -1)

In [11]:
from sklearn.metrics import accuracy_score,confusion_matrix, precision_score, recall_score, f1_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold, GridSearchCV


import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier,HistGradientBoostingClassifier,\
                                StackingClassifier,VotingClassifier,IsolationForest,\
                                RandomForestRegressor
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB,BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.naive_bayes import CategoricalNB
from xgboost import XGBClassifier
# import lightgbm as lgb
from sklearn.tree import DecisionTreeClassifier

from sklearn.gaussian_process.kernels import RBF
from sklearn.model_selection import cross_val_predict, RandomizedSearchCV,GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, cohen_kappa_score, matthews_corrcoef
from sklearn.model_selection import KFold
import pandas as pd


#Scale the data
SS1= StandardScaler()
X_scaled = SS1.fit_transform(X_reshape)


models = {
    'XGBoost': XGBClassifier(),
    'RandomForest': RandomForestClassifier(),
    'HistGradientBoosting': HistGradientBoostingClassifier(),
    'SVM': SVC(),
    'GradientBoosting': GradientBoostingClassifier(),
    'KNeighbors': KNeighborsClassifier(),
    'MLP': MLPClassifier(),
    'DecisionTree': DecisionTreeClassifier(),
    # 'MultinomialNB': MultinomialNB(),
    'GaussianNB': GaussianNB(),
    'BernoulliNB': BernoulliNB(),
    'LogisticRegression': LogisticRegression(),
    'AdaBoost': AdaBoostClassifier(),
    # 'Bagging': BaggingClassifier(),
    'ExtraTrees': ExtraTreesClassifier(),

}


def predict_binary(model, X, threshold=0.5):
    """
    Predicts binary labels for the given input X using the specified model.
    Predictions are determined based on the specified threshold.

    Parameters:
    - model: Trained Keras model for prediction
    - X: Input data for prediction
    - threshold: Threshold for converting probabilities to binary labels

    Returns:
    - Binary labels (0 or 1) for each input sample
    """
    # Get the model's probability predictions for the input data
    prob_predictions = model.predict(X)

    # Apply threshold to convert probabilities to binary labels
    binary_predictions = (prob_predictions > threshold).astype(int)

    return binary_predictions





def evaluate_model(model, X, y, n_splits):
    kf = KFold(n_splits=n_splits,shuffle=True,random_state=42)
    accuracies, precisions, recalls,f1s,cm,specificitys, sensitivitys,roc_aucs,kappas,mccs= [], [], [],[],[],[],[],[],[],[]

    for train_idx, test_idx in kf.split(X):
      # take the location of the splited data and access it by the index

      X_train, y_train = (X.iloc[train_idx], y.iloc[train_idx]) if isinstance(X, pd.DataFrame) else (X[train_idx], y[train_idx])
      X_test, y_test = (X.iloc[test_idx], y.iloc[test_idx]) if isinstance(X, pd.DataFrame) else (X[test_idx], y[test_idx])
      # train the model
      model = build_sequential_model(input_shape=scaled_data.shape[1:])
      model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data = (X_test, y_test), shuffle=True)
      # make prediction
      # Example of using the custom prediction function
      y_pred = predict_binary(model, X_test)

      # evaluation scores
      accuracy = accuracy_score(y_test, y_pred)
      precision = precision_score(y_test, y_pred,zero_division=1)
      recall = recall_score(y_test, y_pred)
      f1= f1_score(y_test, y_pred)
      cms = confusion_matrix(y_test, y_pred)  # Calculate confusion matrix
      roc_auc=roc_auc_score(y_test, y_pred)

      # calculate the specificity and sensitivity
      tn, fp, fn, tp = cms.ravel()
      specificity = tn / (tn + fp)
      sensitivity = tp / (tp + fn)
      # Calculate Cohen's Kappa
      kappa = cohen_kappa_score(y_test, y_pred)
      mcc = matthews_corrcoef(y_test, y_pred)





      # append all the results
      accuracies.append(accuracy)
      precisions.append(precision)
      recalls.append(recall)
      f1s.append(f1)
      cm.append(cms)  # Add confusion matrix to list
      specificitys.append(specificity) #add specificity score
      sensitivitys.append(sensitivity) #add sensitivity scores to the list
      roc_aucs.append(roc_auc) #add roc_auc score to the list
      kappas.append(kappa)
      mccs.append(mcc)


    return (format(sum(accuracies)/n_splits, '.3f'), format(sum(precisions)/n_splits, '.3f'),
            format(sum(recalls)/n_splits, '.3f'),format(sum(f1s)/n_splits, '.3f'),format(sum(roc_aucs)/n_splits, '.3f'), sum(cm)/n_splits ,format(sum(specificitys)/n_splits, '.3f'),
            format(sum(sensitivitys)/n_splits, '.3f'),format(sum(kappas)/n_splits, '.3f'),format(sum(mccs)/n_splits, '.3f'))




def modelApplied(Model,X_train , y_train, n_splits=5,MN='model_name',model_result = pd.DataFrame()):
  accuracy, precision, recall,f1,roc_auc,cm,specificity,sensitivity,kappa,mcc = evaluate_model(Model,X_train , y_train, n_splits)

  new_row=[MN,accuracy,precision,recall,f1,roc_auc,specificity, sensitivity,kappa,mcc]

  model_result.loc[len(model_result)]=new_row
  print(new_row)
  print('DONE!!')

In [ ]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>ALL band<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
# ['XGBoost', '0.957', '0.987', '0.926', '0.955', '0.957', '0.987', '0.926', '0.913', '0.915']
# DONE!!
# ['RandomForest', '0.937', '0.973', '0.898', '0.934', '0.937', '0.975', '0.898', '0.873', '0.876']
# DONE!!
# ['HistGradientBoosting', '0.958', '0.991', '0.925', '0.957', '0.958', '0.991', '0.925', '0.916', '0.919']
# DONE!!
# ['SVM', '0.916', '0.960', '0.868', '0.911', '0.916', '0.963', '0.868', '0.831', '0.835']
# DONE!!
# ['GradientBoosting', '0.942', '0.975', '0.907', '0.940', '0.942', '0.977', '0.907', '0.884', '0.886']
# DONE!!
# ['KNeighbors', '0.818', '0.988', '0.644', '0.778', '0.818', '0.992', '0.644', '0.636', '0.679']
# DONE!!
# ['MLP', '0.928', '0.953', '0.901', '0.926', '0.928', '0.955', '0.901', '0.856', '0.857']
# DONE!!
# ['DecisionTree', '0.891', '0.887', '0.897', '0.891', '0.891', '0.885', '0.897', '0.782', '0.782']
# DONE!!
# ['GaussianNB', '0.857', '0.985', '0.726', '0.836', '0.857', '0.989', '0.726', '0.714', '0.741']
# DONE!!
# ['BernoulliNB', '0.865', '0.933', '0.787', '0.854', '0.865', '0.943', '0.787', '0.731', '0.740']
# DONE!!
# ['LogisticRegression', '0.899', '0.911', '0.885', '0.897', '0.899', '0.913', '0.885', '0.798', '0.798']
# DONE!!
# ['AdaBoost', '0.923', '0.947', '0.897', '0.921', '0.923', '0.950', '0.897', '0.847', '0.848']
# DONE!!
# ['ExtraTrees', '0.929', '0.964', '0.892', '0.926', '0.929', '0.967', '0.892', '0.858', '0.861']
# DONE!!

#Optimization

In [26]:
def signal_normalize(signal):
  X_merged_features_axis = np.moveaxis(signal,1,2)
  X_merged_features_reshaped = X_merged_features_axis.reshape(X_merged_features_axis.shape[0],-1)
  scaler= MinMaxScaler(feature_range=(0, 1))
  scaled_data_reshaped = scaler.fit_transform(X_merged_features_reshaped)

# Reshape back to original shape if necessary
  scaled_data = scaled_data_reshaped.reshape(X_merged_features_axis.shape)
  return scaled_data

In [12]:

from sklearn.preprocessing import StandardScaler, MinMaxScaler
import numpy as np



X_merged_features_axis = np.moveaxis(X_merged_features,1,2)
X_merged_features_reshaped = X_merged_features_axis.reshape(X_merged_features_axis.shape[0],-1)
# scaler = StandardScaler()
scaler= MinMaxScaler(feature_range=(0, 1))
scaled_data_reshaped = scaler.fit_transform(X_merged_features_reshaped)

# Reshape back to original shape if necessary
scaled_data = scaled_data_reshaped.reshape(X_merged_features_axis.shape)

In [13]:
scaled_data.shape

(5834, 37, 29)

In [27]:
#for MFCC features
X_scaled_mfcc = signal_normalize(X_mfcc)

In [28]:
from sklearn.model_selection import train_test_split
xtrian_mfcc,xtest_mfcc, ytrain_mfcc, ytest_mfcc = train_test_split(X_scaled_mfcc,Y,test_size= 0.2,shuffle= True, random_state=42)

In [ ]:
from sklearn.model_selection import train_test_split
xtrian,xtest, ytrain, ytest = train_test_split(scaled_data,Y,test_size= 0.2,shuffle= True, random_state=42)

In [15]:
# from wandb.keras import WandbCallback
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import callbacks
from tensorflow.keras.backend import clear_session
from tensorflow.keras.layers import  Conv1D,Conv1DTranspose,BatchNormalization,LeakyReLU,MaxPool1D,Flatten,\
GlobalAveragePooling1D,Dense,Dropout,AveragePooling1D,ReLU,MaxPooling1D,Conv2D,MaxPool2D,GlobalAveragePooling2D,\
AveragePooling2D,Bidirectional, LSTM, concatenate,Reshape
from tensorflow.keras.regularizers import l2
from keras.layers import ConvLSTM1D, LSTM

from tensorflow.keras.models import Sequential
from tensorflow.keras.backend import clear_session
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import LSTM

In [16]:
%%capture
!pip install tensorflow_addons

In [33]:
%%time
import tensorflow_addons as tfa
from keras.optimizers import RMSprop, Adam
def build_sequential_model(input_shape):

    clear_session()
    model = Sequential()


    model.add(Conv1D(filters=256, kernel_size=5,strides =1, activation='relu',input_shape=input_shape))
    model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
    model.add(Conv1D(filters=64, kernel_size=5, activation='relu'))
    # model.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
    # model.add(Conv1D(filters=64, kernel_size=5, activation='relu'))
    model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
    model.add(Conv1D(filters=256, kernel_size=5, activation='relu'))




    # model.add(MaxPool1D(pool_size=4,padding = 'same'))
    # model.add(MaxPool1D(pool_size=4,padding = 'same'))
    model.add(MaxPool1D(pool_size=4,padding = 'same'))
    model.add(Dropout(0.25))


    # Flatten for Dense layers after Conv1D/LSTM processing

    model.add(Flatten())
    # model.add(Dense(1024, activation='relu'))
    # model.add(Dense(256, activation='relu'))
    # model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    # opt = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    opt = tfa.optimizers.RectifiedAdam(learning_rate=0.001)
    opt = tfa.optimizers.SWA(opt)
    model.compile(optimizer = opt,
        # loss=focal_loss(),
                  loss= 'binary_crossentropy',
        metrics=['accuracy'])
    model.summary()
    return model
latent_dim = 29
Model = build_sequential_model(input_shape=xtrian_mfcc.shape[1:])
Model.fit(xtrian_mfcc,ytrain_mfcc, epochs=50, batch_size=64, validation_data = (xtest_mfcc, ytest_mfcc
                                                                      ), shuffle=True)#

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 22, 256)           37376     
                                                                 
 conv1d_1 (Conv1D)           (None, 18, 128)           163968    
                                                                 
 conv1d_2 (Conv1D)           (None, 14, 64)            41024     
                                                                 
 conv1d_3 (Conv1D)           (None, 10, 128)           41088     
                                                                 
 conv1d_4 (Conv1D)           (None, 6, 256)            164096    
                                                                 
 max_pooling1d (MaxPooling1  (None, 2, 256)            0         
 D)                                                              
                                                        

In [20]:
Model.fit(xtrian,ytrain, epochs=50, batch_size=64, validation_data = (xtest, ytest
                                                                      ), shuffle=True)#

Epoch 1/50
73/73 [==============================] - 6s 87ms/step - loss: 0.0540 - accuracy: 0.9816 - val_loss: 0.2099 - val_accuracy: 0.9374
Epoch 2/50
73/73 [==============================] - 5s 67ms/step - loss: 0.0500 - accuracy: 0.9835 - val_loss: 0.2256 - val_accuracy: 0.9212
Epoch 3/50
73/73 [==============================] - 5s 66ms/step - loss: 0.0443 - accuracy: 0.9865 - val_loss: 0.2294 - val_accuracy: 0.9400
Epoch 4/50
73/73 [==============================] - 6s 86ms/step - loss: 0.0416 - accuracy: 0.9859 - val_loss: 0.2384 - val_accuracy: 0.9434
Epoch 5/50
73/73 [==============================] - 5s 67ms/step - loss: 0.0405 - accuracy: 0.9869 - val_loss: 0.2226 - val_accuracy: 0.9434
Epoch 6/50
73/73 [==============================] - 6s 85ms/step - loss: 0.0508 - accuracy: 0.9820 - val_loss: 0.2283 - val_accuracy: 0.9272
Epoch 7/50
73/73 [==============================] - 5s 67ms/step - loss: 0.0369 - accuracy: 0.9865 - val_loss: 0.2430 - val_accuracy: 0.9469
Epoch 8/50
73

In [ ]:

all_band_result = pd.DataFrame(columns=['Models', 'Accuracy', 'Precision', 'Recall', 'F1-Score', 'Roc_Auc', 'Specificity', 'Sensitivity', 'Kappa', 'MCC'])

print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>ALL band<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")

modelApplied(Model, scaled_data, Y, MN='CNN',model_result= all_band_result)

In [ ]:
 ['CNN', '0.965', '0.977', '0.952', '0.964', '0.965', '0.977', '0.952', '0.929', '0.930']
DONE!!
['CNN', '0.922', '0.950', '0.891', '0.919', '0.921', '0.952', '0.891', '0.843', '0.846']
DONE!!

SyntaxError: invalid syntax (<ipython-input-46-48cce0cbbba1>, line 2)

In [ ]:
from sklearn.model_selection import KFold
import numpy as np

# Configuration
n_splits = 5  # Number of folds
random_state = 42
shuffle = True

# Initialize KFold

kf = KFold(n_splits=n_splits, shuffle=shuffle, random_state=random_state)

# Create a placeholder array for fold assignments
fold_assignments = np.empty(len(Y), dtype=int)

# Split the data using kf.split. Note: kf.split does not need Y for splitting, but it's kept here for consistency
for fold, (train_idx, test_idx) in enumerate(kf.split(scaled_data, Y)):
    # Assign fold number
    fold_assignments[test_idx] = fold

# Example: To get the train and test sets for the first fold
fold_number = 0  # Choose which fold to use (0 to n_splits-1)
train_idx = np.where(fold_assignments != fold_number)[0]
test_idx = np.where(fold_assignments == fold_number)[0]

X_train, X_test = scaled_data[train_idx], scaled_data[test_idx]
y_train, y_test = Y[train_idx], Y[test_idx]

In [ ]:
Model.fit(X_train, y_train, epochs=1, batch_size=128, validation_data = (X_test, y_test), shuffle=True)

73/73 [==============================] - 15s 208ms/step - loss: 0.0292 - accuracy: 0.9891 - val_loss: 0.3378 - val_accuracy: 0.9306


In [ ]:

# make prediction
y_pred = Model.predict(X_test)
def predict_binary(model, X, threshold=0.5):
    """
    Predicts binary labels for the given input X using the specified model.
    Predictions are determined based on the specified threshold.

    Parameters:
    - model: Trained Keras model for prediction
    - X: Input data for prediction
    - threshold: Threshold for converting probabilities to binary labels

    Returns:
    - Binary labels (0 or 1) for each input sample
    """
    # Get the model's probability predictions for the input data
    prob_predictions = model.predict(X)

    # Apply threshold to convert probabilities to binary labels
    binary_predictions = (prob_predictions > threshold).astype(int)

    return binary_predictions

# Example of using the custom prediction function
y_pred_binary = predict_binary(Model, X_test)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred_binary)
precision = precision_score(y_test, y_pred_binary, zero_division=1)
recall = recall_score(y_test, y_pred_binary)
f1 = f1_score(y_test, y_pred_binary)
roc_auc = roc_auc_score(y_test, y_pred_binary)
conf_matrix = confusion_matrix(y_test, y_pred_binary)
tn, fp, fn, tp = conf_matrix.ravel()
specificity = tn / (tn+fp)
sensitivity = tp / (tp+fn)
kappa = cohen_kappa_score(y_test, y_pred_binary)
mcc = matthews_corrcoef(y_test, y_pred_binary)

# Print the results
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC AUC Score: {roc_auc:.4f}")
print(f"Confusion Matrix:\n {conf_matrix}")
print(f"Specificity: {specificity:.4f}")
print(f"Sensitivity: {sensitivity:.4f}")
print(f"Cohen's Kappa: {kappa:.4f}")
print(f"Matthews Correlation Coefficient: {mcc:.4f}")

37/37 [==============================] - 1s 35ms/step
Accuracy: 0.9306
Precision: 0.9922
Recall: 0.8688
F1 Score: 0.9264
ROC AUC Score: 0.9310
Confusion Matrix:
 [[576   4]
 [ 77 510]]
Specificity: 0.9931
Sensitivity: 0.8688
Cohen's Kappa: 0.8613
Matthews Correlation Coefficient: 0.8681


#Final Model Exicuition

In [ ]:
X_time.shape, X_frequency.shape, X_merged_features.shape #3 different data is available for analysis

((5834, 29, 18), (5834, 29, 19), (5834, 29, 37))

##Model run without data normalization

###time domain data

data : X_time, X_frequecy, X_merged_features

In [ ]:
from sklearn.model_selection import train_test_split
X_time = X_time.reshape(X_time.shape[0],-1)
X_train, X_test, Y_train,Y_test = train_test_split(X_time,Y, random_state= 42, shuffle = True,test_size=0.20)
X_train.shape, X_test.shape, Y_train.shape,Y_test.shape

((4667, 522), (1167, 522), (4667,), (1167,))

In [ ]:
pd.DataFrame(Y_test).value_counts()

1    587
0    580
dtype: int64

In [ ]:
# total_Metics = []
# total_Metics = pd.DataFrame(total_Metics)
# total_Metics['Classifier'] = 'Classifier'
# total_Metics['Accuracy'] = 'Accuracy'
# total_Metics['mcc'] = 'mcc'
# # total_Metics['auc'] = 'auc'
# total_Metics['Kappa'] = 'Kappa'
# total_Metics['precision'] = 'precision'
# total_Metics['recall'] = 'recall'
# total_Metics['f1'] = 'f1'
# total_Metics['sensitivity'] = 'sensitivity'
# total_Metics['specificity'] = 'specificity'

# cv = KFold(n_splits=5, random_state=42, shuffle=True)

# # create model
# models = [RandomForestClassifier(n_estimators = 450, max_depth = 9),
#           DecisionTreeClassifier(max_depth = 5),
#           ExtraTreesClassifier(n_estimators = 450, max_depth = 7),
#           KNeighborsClassifier(n_neighbors=5),
#           AdaBoostClassifier(n_estimators = 350, learning_rate = 0.5, random_state = 50),
#           XGBClassifier(n_estimators = 350,max_depth = 7, base_score = 0.5, learning_rate = 0.1),
#           # LGBMClassifier(learning_rate = 0.1,max_depth = 7,random_state = 50)
#           ]
# for model in models:
#   from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
#   # evaluate model
#   # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
#   # model.fit(xtrain, ytrain)
#   # pred = model.predict(xtest)
#   pred = cross_val_predict(model, X_time, Y, cv=cv, n_jobs=-1)

#   # cm1 = confusion_matrix(y, y_pred)
#   # report performance
#   Accuracy = accuracy_score(Y, pred)
#   mcc = matthews_corrcoef(Y, pred)
#   cm1 = confusion_matrix(Y, pred)
#   kappa = cohen_kappa_score(Y, pred)
#   f1 = f1_score(Y, pred)
#   precision_score = precision_score(Y, pred)
#   recall_score = recall_score(Y, pred)
#   sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
#   specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
#   # y_pred = np.argmax(y_pred, axis=0)
#   # auc = roc_auc_score(y, y_pred, multi_class='ovr')
#   total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

# print(total_Metics)
# # total_Metics.to_csv("total metrics(FT-CV).csv")

                                          Classifier  Accuracy       mcc  \
0  RandomForestClassifier(max_depth=9, n_estimato...  0.930065  0.864236   
1                DecisionTreeClassifier(max_depth=5)  0.902811  0.809774   
2  ExtraTreesClassifier(max_depth=7, n_estimators...  0.897326  0.797470   
3                             KNeighborsClassifier()  0.775283  0.556909   
4  AdaBoostClassifier(learning_rate=0.5, n_estima...  0.926980  0.856534   
5  XGBClassifier(base_score=0.5, booster=None, ca...  0.945149  0.893378   

      Kappa  precision    recall        f1  sensitivity  specificity  
0  0.860130   0.976453  0.881385  0.926486     0.978745     0.881385  
1  0.805622   0.948131  0.852245  0.897635     0.953377     0.852245  
2  0.794652   0.933757  0.855331  0.892825     0.939321     0.855331  
3  0.550566   0.824052  0.700034  0.756997     0.850531     0.700034  
4  0.853960   0.962839  0.888241  0.924037     0.965718     0.888241  
5  0.890298   0.985421  0.903668  0.94277

In [ ]:
time_domain_result = pd.DataFrame(columns=['Models', 'Accuracy', 'Precision', 'Recall', 'F1-Score', 'Roc_Auc', 'Specificity', 'Sensitivity', 'Kappa', 'MCC'])
for k,v in models.items():
  modelApplied(v,X_time , Y, n_splits=5,MN=k,model_result = time_domain_result)

['XGBoost', '0.942', '0.980', '0.903', '0.940', '0.942', '0.981', '0.903', '0.884', '0.887']
DONE!!
['RandomForest', '0.935', '0.974', '0.895', '0.933', '0.935', '0.976', '0.895', '0.871', '0.874']
DONE!!
['HistGradientBoosting', '0.943', '0.986', '0.899', '0.940', '0.943', '0.987', '0.899', '0.886', '0.889']
DONE!!
['SVM', '0.791', '0.864', '0.691', '0.768', '0.791', '0.891', '0.691', '0.582', '0.594']
DONE!!
['GradientBoosting', '0.926', '0.962', '0.888', '0.923', '0.926', '0.964', '0.888', '0.853', '0.855']
DONE!!
['KNeighbors', '0.775', '0.825', '0.700', '0.756', '0.775', '0.851', '0.700', '0.551', '0.558']
DONE!!


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


['MLP', '0.832', '0.843', '0.825', '0.831', '0.831', '0.837', '0.825', '0.663', '0.669']
DONE!!
['DecisionTree', '0.889', '0.888', '0.889', '0.889', '0.889', '0.888', '0.889', '0.777', '0.777']
DONE!!
['GaussianNB', '0.771', '0.877', '0.630', '0.733', '0.771', '0.912', '0.630', '0.542', '0.565']
DONE!!
['BernoulliNB', '0.578', '0.577', '0.581', '0.579', '0.578', '0.575', '0.581', '0.156', '0.156']
DONE!!


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

['LogisticRegression', '0.811', '0.828', '0.787', '0.806', '0.811', '0.836', '0.787', '0.623', '0.624']
DONE!!
['AdaBoost', '0.914', '0.941', '0.883', '0.911', '0.914', '0.944', '0.883', '0.828', '0.829']
DONE!!
['ExtraTrees', '0.932', '0.968', '0.892', '0.929', '0.932', '0.971', '0.892', '0.863', '0.866']
DONE!!


In [ ]:






Classifier = {
    'XGBoost': XGBClassifier(n_estimators=500, learning_rate=.1, max_depth=5,base_score = 0.5),
    'RandomForest': RandomForestClassifier(n_estimators=150,criterion = 'gini',max_features= 'sqrt'),
    'HistGradientBoosting': HistGradientBoostingClassifier( max_leaf_nodes=20,max_iter=300,max_depth=17),
    'SVM': SVC(),
    'GradientBoosting': GradientBoostingClassifier(learning_rate= .2,n_estimators=450,),
    'KNeighbors': KNeighborsClassifier(),
    'MLP': MLPClassifier(),
    'DecisionTree': DecisionTreeClassifier(),
    # 'MultinomialNB': MultinomialNB(),
    'GaussianNB': GaussianNB(),
    'BernoulliNB': BernoulliNB(),
    'LogisticRegression': LogisticRegression(),
    'AdaBoost': AdaBoostClassifier(estimator = RandomForestClassifier(n_estimators=150,criterion = 'gini',max_features= 'sqrt')),
    # 'Bagging': BaggingClassifier(),
    'ExtraTrees': ExtraTreesClassifier(n_estimators=150),

}

time_domain_result = pd.DataFrame(columns=['Models', 'Accuracy', 'Precision', 'Recall', 'F1-Score', 'Roc_Auc', 'Specificity', 'Sensitivity', 'Kappa', 'MCC'])
for k,v in Classifier.items():
  modelApplied(v,X_time , Y, n_splits=5,MN=k,model_result = time_domain_result)

['XGBoost', '0.944', '0.982', '0.905', '0.942', '0.944', '0.983', '0.905', '0.888', '0.891']
DONE!!
['RandomForest', '0.933', '0.970', '0.894', '0.930', '0.933', '0.972', '0.894', '0.866', '0.869']
DONE!!
['HistGradientBoosting', '0.943', '0.985', '0.900', '0.940', '0.943', '0.986', '0.900', '0.886', '0.890']
DONE!!
['SVM', '0.791', '0.864', '0.691', '0.768', '0.791', '0.891', '0.691', '0.582', '0.594']
DONE!!


KeyboardInterrupt: 